# Electricity Usage Prediction

## Introduction

In a world where sustainable development goals (SGD) are of paramount importance, it is crucial for stakeholders to predict the energy usage in advance so as to maximise energy utilisation. This project attempts to address this problem by regressing energy usage against weather conditions, a feature that should provide an abundance of information towards electricity consumption.

## Preliminary Work

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from datetime import datetime

C:\Users\AU YONG Chun Sang\miniconda3\lib\site-packages\torch\package\_directory_reader.py:17: UserWarning: Failed to initialize NumPy: module compiled against API version 0xe but this version of numpy is 0xd (Triggered internally at  C:\Users\builder\tkoch\workspace\pytorch\pytorch_1647970138273\work\torch\csrc\utils\tensor_numpy.cpp:68.)
  _dtype_to_storage = {data_type(0).dtype: data_type for data_type in _storages}


In [3]:
energy = pd.read_csv('energy_data.csv')
weather = pd.read_csv('weather_data.csv')

The records in energy_data.csv are measured half-hourly. We first sum up the energy usage to get per day usage and merge it with weather data.

In [11]:
def getdate(datetime):
    return datetime.split()[0]
energy['Date'] = energy['Date & Time'].apply(getdate)

In [13]:
energy.groupby('Date').sum()

,use [kW],gen [kW],Grid [kW],AC [kW],Furnace [kW],Cellar Lights [kW],Washer [kW],First Floor lights [kW],Utility Rm + Basement Bath [kW],Garage outlets [kW],MBed + KBed outlets [kW],Dryer + egauge [kW],Panel GFI (central vac) [kW],Home Office (R) [kW],Dining room (R) [kW],Microwave (R) [kW],Fridge (R) [kW]
Date,,,,,,,,,,,,,,,,,
2014-01-01,65.013592,0.0,65.013592,0.042977,8.814319,1.137579,0.750298,0.567603,0.178529,0.261094,0.254839,31.938131,0.350291,3.272944,0.200970,4.997037,4.639598
2014-01-02,32.305336,0.0,32.305336,0.047452,10.830045,0.600321,0.323182,0.506440,0.178024,0.282479,0.798316,5.423866,0.346679,3.475469,0.207041,1.534426,3.881399
2014-01-03,31.164468,0.0,31.164468,0.055865,12.417151,0.442453,0.004276,0.507426,0.176649,0.279159,0.746972,0.005554,0.344061,3.615520,0.201975,1.667553,3.671391
2014-01-04,45.287782,0.0,45.287782,0.048827,11.147332,0.674477,1.046294,0.515988,0.180056,0.344005,0.640721,19.994908,0.346872,3.700408,0.203913,1.029198,3.357907
2014-01-05,36.316643,0.0,36.316643,0.039831,9.301135,0.686189,0.235143,0.519449,0.178556,0.348489,0.584570,9.493912,0.346070,3.699178,0.197897,1.619991,4.373730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-27,35.046127,0.0,35.046127,0.029014,4.306682,0.475110,0.019649,0.825872,0.112661,0.255620,5.422751,0.005953,0.015400,0.473471,0.668127,0.642506,3.839653
2014-12-28,37.695824,0.0,37.695824,0.044794,4.640888,0.456018,0.035832,0.874563,0.115673,0.257369,11.602281,0.008270,0.018872,0.473571,0.657405,0.311556,3.510436
2014-12-29,28.675929,0.0,28.675929,0.037950,6.143640,0.463515,0.020115,0.552812,0.112204,0.274396,5.951963,0.005461,0.015199,0.493595,0.670818,0.279923,3.702587


In [15]:
weather

,temperature,icon,humidity,visibility,summary,pressure,windSpeed,cloudCover,time,windBearing,precipIntensity,dewPoint,precipProbability
0,34.98,partly-cloudy-night,0.64,10.00,Partly Cloudy,1017.69,7.75,0.29,1388534400,279,0.0,23.89,0.0
1,16.49,clear-night,0.62,10.00,Clear,1022.76,2.71,0.06,1388538000,195,0.0,5.87,0.0
2,14.63,clear-night,0.68,10.00,Clear,1022.32,4.84,0.03,1388541600,222,0.0,6.17,0.0
3,13.31,clear-night,0.71,10.00,Clear,1021.64,4.00,0.14,1388545200,209,0.0,5.63,0.0
4,13.57,clear-night,0.71,9.93,Clear,1020.73,3.67,0.04,1388548800,217,0.0,5.87,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,27.48,clear-day,0.35,10.00,Clear,1023.54,10.54,0.24,1420052400,311,0.0,3.21,0.0
8756,27.17,partly-cloudy-day,0.35,10.00,Partly Cloudy,1023.60,9.53,0.25,1420056000,297,0.0,3.45,0.0
8757,25.72,clear-day,0.37,10.00,Clear,1023.44,8.12,0.08,1420059600,292,0.0,3.20,0.0
8758,22.75,clear-night,0.42,10.00,Clear,1023.29,4.43,0.05,1420063200,299,0.0,3.13,0.0


In [25]:
print(datetime.utcfromtimestamp(1388538000).strftime('%Y-%m-%d %H:%M:%S'))

2014-01-01 01:00:00


## Linear Regression

With the data set ready, we will now train a linear regression model, and then predict energy usage for each day in the month of December using features from weather data.

In [31]:
class LinearRegression(torch.nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = torch.nn.Linear(1, 1)  # One in and one out
 
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

regressor = LinearRegression()

In [33]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(regressor.parameters(), lr = 0.01)

In [ ]:
for epoch in range(500):
 
    # Forward pass: Compute predicted y by passing
    # x to the model
    pred_y = regressor(x_data)
 
    # Compute and print loss
    loss = criterion(pred_y, y_data)
 
    # Zero gradients, perform a backward pass,
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('epoch {}, loss {}'.format(epoch, loss.item()))

After training the model, we evaluate its performance using root MSE loss on December data.

In [ ]:
december_pred = regressor(december_x)
loss = np.sqrt(torch.nn.functional.mse_loss(december_pred, december_target))

In [34]:
# create csv dump